<a href="https://colab.research.google.com/github/hobo-hobo/InternshipAmsterdamGeneral/blob/master/Inspect_Prediction_Differences.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inspect Differences

## Installing and loading

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import nltk

In [ ]:
from nltk.corpus import stopwords
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
freq_dict = dict()

with open ("/content/frequency_merge_wiki_child.txt", "r") as infile:
  data = infile.readlines()

for line in data:
  line = line.strip()
  info = line.split(" ")
  word = info [0]
  freq = int(info [1])
  freq_dict[word]=freq


In [ ]:
with open ("/content/lex.mturk.txt","r",encoding="ISO-8859-1") as infile:
  lexmturk = infile.readlines()[1:]

annotations = []
for line in lexmturk:
  info = line.strip().split("\t")
  sentence = info[0]
  cword = info[1]
  line_annotations = [] 
  for an in  info[2:]:
    if an.count(" ")<1:
      line_annotations.append(an)
  annotations.append(set(line_annotations))

## Some Functions

In [ ]:
def get_characteristics(row):
  row = row.strip().split("\t")
  sentence = row[0]
  complex_word = row[1]
  options = set(row[2:])
  
  return(sentence, complex_word, options)

In [ ]:
def get_comparisons(base_model, other_model, annotations):
  all_model_preds = []
  all_base_preds = []

  ABM = []
  AM = []
  AB = []
  BM = []
  A = []
  B = []
  M = []

  len_ABM = []
  len_AM = []
  len_AB = []
  len_BM = []
  len_A = []
  len_B = []
  len_M = []

  sents = []
  cwords = []

  for base_line, model_line, annotations_line in zip(base_model, other_model, annotations):
    if len(base_line)>3:
      sent, complex_word, base_options = get_characteristics(base_line)
    else: continue
    if len(model_line)>3:
      sent, complex_word, model_options = get_characteristics(model_line)
    else: continue
    
    all_model_preds.append(model_options)
    all_base_preds.append(base_options)

    ABM_predictions = base_options.intersection(model_options, annotations_line)
    AM_predictions = annotations_line.intersection(model_options).difference(base_options)
    AB_predictions = annotations_line.intersection(base_options).difference(model_options)
    BM_predictions = base_options.intersection(model_options).difference(annotations_line)
    A_predictions = annotations_line.difference(base_options, model_options)
    B_predictions = base_options.difference(model_options, annotations_line)
    M_predictions = model_options.difference(base_options, annotations_line)

    all_preds = base_options.union(model_options, annotations_line)

    ABM.append(ABM_predictions)
    AM.append(AM_predictions)
    AB.append(AB_predictions)
    BM.append(BM_predictions)
    A.append(A_predictions)
    B.append(B_predictions)
    M.append(M_predictions)


    len_ABM.append(len(ABM_predictions))
    len_AM.append(len(AM_predictions))
    len_AB.append(len(AB_predictions))
    len_BM.append(len(BM_predictions))
    len_A.append(len(A_predictions))
    len_B.append(len(B_predictions))
    len_M.append(len(M_predictions))

    sents.append(sent)
    cwords.append(complex_word)  

  return all_base_preds, all_model_preds, ABM, AM, AB, BM, A, B, M, len_ABM, len_AM, len_AB, len_BM, len_A, len_B, len_M, sents, cwords,

In [ ]:
def get_frequencies(words):
  # word_list = words
  word_list = [item for wordset in words for item in wordset]
  total_freq = 0
  frequencies = []
  for word in word_list:
    if word in freq_dict.keys():
      # print(word)
      freq = freq_dict[word]
      total_freq += freq
      frequencies.append(freq)
  
  return(frequencies)

## Analysis

In [ ]:
wwm_path = "/content/LMT_WWM_outputs.txt"
with open (wwm_path, "r") as infile:
  base_model = infile.readlines()  

In [ ]:
stop_words = set(stopwords.words('english'))

In [ ]:
other_model_paths = ["/content/LMT_FT_2class_10000_sents_outputs.txt", "/content/LMT_FT_2class_50000_lrtiny_outputs.txt", "/content/LMT_FT_mlm_1000_lr_5e-6_outputs.txt", "/content/LMT_FT_mlm_1000_sents_outputs.txt", "/content/LMT_FT_mlm_50000_5e-7_outputs.txt"] 
with open("numbers.csv","a") as outfile:
  outfile.write("setting\t occurence\t avg_tok_len\t median\t sigma\t stop_word_count\t stop_word_unique\n")

  for model in other_model_paths:
    model_name = model.replace("/content/","")
    print("\nNumbers for model:", model_name)
    outfile.write(model_name)
    
    with open (model, "r") as infile:
      other = infile.readlines()
      comparisons = get_comparisons(base_model, other, annotations)

      for setting, name in zip(comparisons[:9], ["all_base_preds", "all_model_preds", "ABM", "AM", "AB", "BM", "A", "B", "M"]):
        print("\nSetting:",name)
        
        frequencies = get_frequencies(setting)
        occurence = len(frequencies)

        print("This setting occured", occurence, "times")

        if occurence> 1:
        
          flattened =  [item for wordset in setting for item in wordset]
          lengths = [len(i) for i in flattened]
          
          stop_word_count = 0  
          for stop_word in stop_words:
            stop_word_count += flattened.count(stop_word)

          stop_word_unique = len(stop_words.intersection(set(flattened)))

          avg_tok_len = sum(lengths) / len(lengths)
          median = statistics.median(frequencies)
          sigma = statistics.stdev(frequencies)

          outfile.write(name+"\t"+str(occurence)+"\t"+str(avg_tok_len)+"\t"+str(median)+"\t"+str(sigma)+"\t"+str(stop_word_count)+"\t"+str(stop_word_unique)+"\n")
          
        else:
          print("Not enough occurrence")



Numbers for model: LMT_FT_2class_10000_sents_outputs.txt

Setting: all_base_preds
This setting occured 5000 times

Setting: all_model_preds
This setting occured 5000 times

Setting: ABM
This setting occured 1132 times

Setting: AM
This setting occured 438 times

Setting: AB
This setting occured 347 times

Setting: BM
This setting occured 1425 times

Setting: A
This setting occured 3154 times

Setting: B
This setting occured 2096 times

Setting: M
This setting occured 2005 times

Numbers for model: LMT_FT_2class_50000_lrtiny_outputs.txt

Setting: all_base_preds
This setting occured 5000 times

Setting: all_model_preds
This setting occured 4998 times

Setting: ABM
This setting occured 1191 times

Setting: AM
This setting occured 221 times

Setting: AB
This setting occured 288 times

Setting: BM
This setting occured 2059 times

Setting: A
This setting occured 3371 times

Setting: B
This setting occured 1462 times

Setting: M
This setting occured 1527 times

Numbers for model: LMT_FT_mlm_

In [ ]:
comparisons = pd.DataFrame(list(zip(sents, cwords, ABM, len_ABM, AM, len_AM,  AB, len_AB, BM, len_BM, A, len_A, B, len_B, M, len_M)), columns= ["Sentence", "complex word", "ABM", "len ABM", "AM", "len AM",  "AB", "len AB", "BM", "len BM", "A", "len A", "B", "len B", "M", "len M"])
comparisons.to_csv("/content/comparisons.csv",sep=";")